<a href="https://colab.research.google.com/github/ayushi15092002/mental-health-workload/blob/main/data_preprocess_old.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install mne

In [ ]:
import mne
import os
from glob import glob
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
subjectData = "/content/drive/My Drive/drdo/subjects"
cleanData = "/content/drive/My Drive/drdo/cleanData_new"
fname= "/content/drive/My Drive/drdo/channel_loc.csv"
read_dataset(subjectData)

In [ ]:
cleanData = "/content/drive/My Drive/drdo/cleanData_new"
fname= "/content/drive/My Drive/drdo/channel_loc.csv"
path_file_vhdr = '/content/drive/My Drive/drdo/subjects/VP001/nback1.vhdr'
path_file_vmrk = '/content/drive/My Drive/drdo/subjects/VP001/nback1.vmrk'
preprocess(path_file_vhdr, path_file_vmrk, "VP001", "nback1")

In [ ]:
def read_dataset(subjectPath):
    
    try:
        root, dirs, files = next(os.walk(subjectPath))
        
        for folder_name in dirs:
          path_file_vhdr = subjectPath + '/' + folder_name + '/nback1.vhdr'
          path_file_vmrk = subjectPath + '/' + folder_name + '/nback1.vmrk'
          preprocess(path_file_vhdr, path_file_vmrk, folder_name, "nback1")

          path_file_vhdr = subjectPath + '/' + folder_name + '/nback2.vhdr'
          path_file_vmrk = subjectPath + '/' + folder_name + '/nback2.vmrk'
          preprocess(path_file_vhdr, path_file_vmrk, folder_name, "nback2")

          path_file_vhdr = subjectPath + '/' + folder_name + '/nback3.vhdr'
          path_file_vmrk = subjectPath + '/' + folder_name + '/nback3.vmrk'
          preprocess(path_file_vhdr, path_file_vmrk,folder_name,"nback3")

          # path_file_vhdr = subjectPath + '/' + folder_name + '/gonogo1.vhdr'
          # path_file_vmrk = subjectPath + '/' + folder_name + '/gonogo1.vmrk'
          # preprocess(path_file_vhdr, path_file_vmrk, folder_name, "gonogo1",2)

          # path_file_vhdr = subjectPath + '/' + folder_name + '/gonogo2.vhdr'
          # path_file_vmrk = subjectPath + '/' + folder_name + '/gonogo2.vmrk'
          # preprocess(path_file_vhdr, path_file_vmrk, folder_name, "gonogo2",2)

          # path_file_vhdr = subjectPath + '/' + folder_name + '/gonogo3.vhdr'
          # path_file_vmrk = subjectPath + '/' + folder_name + '/gonogo3.vmrk'
          # preprocess(path_file_vhdr, path_file_vmrk,folder_name,"gonogo3",2)

          # path_file_vhdr = subjectPath + '/' + folder_name + '/word1.vhdr'
          # path_file_vmrk = subjectPath + '/' + folder_name + '/word1.vmrk'
          # preprocess(path_file_vhdr, path_file_vmrk, folder_name, "word1",3)

          # path_file_vhdr = subjectPath + '/' + folder_name + '/word2.vhdr'
          # path_file_vmrk = subjectPath + '/' + folder_name + '/word2.vmrk'
          # preprocess(path_file_vhdr, path_file_vmrk, folder_name, "word2",3)

          # path_file_vhdr = subjectPath + '/' + folder_name + '/word3.vhdr'
          # path_file_vmrk = subjectPath + '/' + folder_name + '/word3.vmrk'
          # preprocess(path_file_vhdr, path_file_vmrk,folder_name,"word3",3)

            
    except StopIteration:
        pass
        print("Error ocurred:")
        print("Directory with dataset does not found!")
        print("Program will be terminated")
        exit(1)

In [ ]:
def preprocess(path_file_vhdr, path_file_vmrk, subject_name , session_name):


  TUB_montage = mne.channels.read_custom_montage(fname)

  # matplotlib.use('Qt5Agg')
  # %matplotlib widget
  # %matplotlib qt
  # %gui qt
  # mne.viz.set_3d_backend("notebook")

  raw = mne.io.read_raw_brainvision(path_file_vhdr, eog=('HEOG', 'VEOG'), preload=True)

  raw.set_montage(TUB_montage)
  # print("raw.info >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>", raw.info)
  raw.plot(start=0, duration=6)

  mrk = mne.read_annotations(path_file_vmrk, sfreq='auto', uint16_codec=None)

  raw_downsampled = raw.copy().resample(sfreq=200)

  raw_filtered = raw_downsampled.copy().filter(l_freq=1 , h_freq= 40)
  raw_filtered.plot()

  raw_notch_filtered = raw_filtered.notch_filter(50, filter_length='auto', phase='zero')
  raw_notch_filtered.plot()  

  raw_re_referenced = mne.set_eeg_reference(raw_notch_filtered,ref_channels='average',copy=True, projection=False)
  finData, times = raw_re_referenced[:]
  finData.plot()

  fig, ax = plt.subplots(2)
  raw.plot_psd(ax=ax[0], show = False, fmax = 60)
  finData.plot_psd(ax=ax[1], show = False, fmax=60)
  ax[0].set_title("PSD before filtering")
  ax[1].set_title("PSD after filtering")
  ax[1].set_xlabel('Frequency(Hz)')
  fig.set_tight_layout(True)
  plt.show()

  n_components = 10 #number of components you want to fit # can be either integer which typically implies number of channels - 1 (if applied average reference)
                  #if floating point number (0-1) fraction of total explained variance
  method = 'fastica'
  max_iter = 100
  fit_params = dict(fastica_it = 5)
  random_state = 42
  ica = mne.preprocessing.ICA(n_components = n_components,
                              method = method,
                              max_iter=max_iter,
                              random_state= random_state
                              )
  ica.fit(finData)

  finData.load_data()
  ica.plot_sources(finData, show_scrollbars=False)
  ica.plot_components(sphere=1)

  #Manual Eye Artifact Removal
  # ica.exclude = [0, 3]  
  # reconst_raw = finData.copy()
  # ica.apply(reconst_raw)
  # finData.plot(title = "finData")
  # reconst_raw.plot(title = "Manual")

  #Automatic (Threshold Based) Eye Artifact Removal
  ica.exclude = []
  reconst_raw = finData.copy()
  # find which ICs match the EOG pattern
  eog_indices, eog_scores = ica.find_bads_eog(finData, threshold = 2.5)
  ica.exclude = eog_indices
  ica.apply(reconst_raw)
  #print(eog_indices)
  reconst_raw.plot(title="Automatic")

  reconst_raw.plot()

  # if dataset == 1:
  events_ids = {
    'Stimulus/S 16': 16,
    'Stimulus/S 48': 48,
    'Stimulus/S 64': 64,
    'Stimulus/S 80': 80,
    'Stimulus/S 96': 96,
    'Stimulus/S112': 112,
    'Stimulus/S128': 128,
    'Stimulus/S144': 144}

  # elif dataset == 2:
  #   events_ids = {
  #     'Stimulus/S 16': 16,
  #     'Stimulus/S 32': 32,
  #     'Stimulus/S 48': 48}

  # elif dataset == 3:
  #   events_ids = {
  #     'Stimulus/S 16': 16,
  #     'Stimulus/S 32': 32}

  
  # events_ids

  event,event_ids = mne.events_from_annotations(reconst_raw)
  print("events >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>", event)
  print("event_ids >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>",event_ids)
  
  mne.viz.plot_events(event,event_id = events_ids, sfreq=raw.info['sfreq'])

  # tmin=-5 # when does the epoch start relative to the event onset # 300ms before the start of the event
  # tmax=60  # when does the event end after the even onset # 500 ms from the start of the event
  # #Is a tuple containing the start of the baseline and end of the baseline
  # baseline= (-5, -2) #None mean begnining of the event and 0 is the start of the event 

  tmin=-0.3 # when does the epoch start relative to the event onset # 300ms before the start of the event
  tmax=1.7  # when does the event end after the even onset # 500 ms from the start of the event

  # #Is a tuple containing the start of the baseline and end of the baseline
  baseline= (None, 0) #None mean begnining of the event and 0 is the start of the event 
  
  epochs = mne.Epochs(reconst_raw, 
                      events=event,
                      event_id=event_ids,
                      tmin=tmin,tmax=tmax, 
                      baseline=baseline,
                      preload=True,event_repeated = 'drop')
  
  epochs.plot_drop_log()

  back_0_list = [mne.EpochsArray(epochs['Stimulus/S 16'], epochs.info), mne.EpochsArray(epochs['Stimulus/S112'], epochs.info)]
  epochs_0_back = mne.concatenate_epochs(back_0_list)
  back_1_list = [mne.EpochsArray(epochs['Stimulus/S 48'], epochs.info), mne.EpochsArray(epochs['Stimulus/S 64'], epochs.info),  mne.EpochsArray(epochs['Stimulus/S128'], epochs.info)]
  epochs_1_back = mne.concatenate_epochs(back_1_list)
  back_2_list = [mne.EpochsArray(epochs['Stimulus/S 80'], epochs.info), mne.EpochsArray(epochs['Stimulus/S 96'], epochs.info),  mne.EpochsArray(epochs['Stimulus/S144'], epochs.info)]
  epochs_2_back = mne.concatenate_epochs(back_2_list)

  epochs.plot(events=event, event_id = event_ids)

  export_dir = cleanData + "/"

  if session_name == "nback1":
    session_id = "1"
  elif session_name == "nback2":
    session_id = "2"
  else:
    session_id = "3"
  try:
    os.makedirs(export_dir)
  except FileExistsError:
    pass
  try:
    back_0_filepath = export_dir + "/0_back" + "/" +subject_name + "_"+session_id +".fif"
    back_1_filepath = export_dir + "/2_back" + "/" +subject_name + "_"+session_id +".fif"
    back_2_filepath = export_dir + "/3_back" + "/" +subject_name + "_"+session_id +".fif"
    mne.Epochs.save(epochs_0_back,fname=back_0_filepath,overwrite=True)
    mne.Epochs.save(epochs_1_back,fname=back_1_filepath,overwrite=True)
    mne.Epochs.save(epochs_2_back,fname=back_2_filepath,overwrite=True)
  except FileExistsError:
    pass